In [29]:
import json
import numpy as np
import pandas as pd
import os

In [30]:
contains_bills = []

for i in range(1, 114):
    path = 'data/' + str(i) + '/bills'  #this path needs to be changed
    if os.path.exists(path):
        contains_bills.append(i)
        
print contains_bills

[109, 112, 113]


In [31]:
def find_subject_in_bills(subject):
    res = []
    for i in range(0, len(tmp)):
        if type(tmp[i]) != list:
            res.append(0)
        else:
            if subject in tmp[i]:
                res.append(1)
            else:
                res.append(0)
    return res

def get_subjects(isAmendment, title):
    if type(title) != str: #
        tmp.append(np.nan)
    else:
        if isAmendment:
            if len(title.split()) > 1:
                bill = title.split()[2]
            else:
                bill = title
        else:
            bill = title
        path = "data/" + str(congress_no) + "/bills/" + ''.join([i for i in bill if not i.isdigit()]) + "/" + bill + "/data.json"
        if os.path.exists(path):
            f = open(path, 'r')
            vote = json.loads(f.read(), 'utf-8')
            if 'subjects' in vote:
                tmp.append(vote['subjects'])
            else:
                tmp.append(np.nan)
        else:
            tmp.append(np.nan)
            
def get_committee(isAmendment, title):
    if type(title) != str: #
        return np.nan
    else:
        if isAmendment:
            if len(title.split()) > 1:
                bill = title.split()[2]
            else:
                bill = title
        else:
            bill = title
        path = "data/" + str(congress_no) + "/bills/" + ''.join([i for i in bill if not i.isdigit()]) + "/" + bill + "/data.json"
        if os.path.exists(path):
            f = open(path, 'r')
            vote = json.loads(f.read(), 'utf-8')
            if len(vote['committees']):
                return vote['committees'][0]['committee']
            else:
                return np.nan
        else:
            return np.nan
        
def get_title(isAmendment, title):
    if type(title) != str: #
        return np.nan
    else:
        if isAmendment:
            if len(title.split()) > 1:
                bill = title.split()[2]
            else:
                bill = title
        else:
            bill = title
        path = "data/" + str(congress_no) + "/bills/" + ''.join([i for i in bill if not i.isdigit()]) + "/" + bill + "/data.json"
        if os.path.exists(path):
            f = open(path, 'r')
            vote = json.loads(f.read(), 'utf-8')
            return vote['short_title']
        else:
            return np.nan
        
def get_sponsor(isAmendment, title):
    if type(title) != str: #
        return np.nan
    else:
        if isAmendment:
            if len(title.split()) > 1:
                bill = title.split()[2]
            else:
                bill = title
        else:
            bill = title
        path = "data/" + str(congress_no) + "/bills/" + ''.join([i for i in bill if not i.isdigit()]) + "/" + bill + "/data.json"
        if os.path.exists(path):
            f = open(path, 'r')
            vote = json.loads(f.read(), 'utf-8')
            if 'sponsor' in vote and vote['sponsor']:
                name = vote['sponsor']['name'].split(', ')
                filtered_name = name[1] + "#" + name[0] + "#" + vote['sponsor']['state']
                return filtered_name
            else:
                return np.nan
        else:
            return np.nan

In [32]:
for congress_no in contains_bills:
    for body in ['house', 'senate']:
        tmp = []
        path = 'csv/' + str(congress_no) + body + '.csv'   #this path needs to be changed
        path2 = 'cleanedcsv/' + str(congress_no) + body + '.csv'  #this path needs to be changed
        df = pd.read_csv(path)
        subjects_dict = {'committee': [], 'billTitle': [], 'sponsor': []}
        for i in range(0, len(df.title)):
            get_subjects(df.isAmendment[i], df.title[i])
            subjects_dict['committee'].append(get_committee(df.isAmendment[i], df.title[i]))
            subjects_dict['billTitle'].append(get_title(df.isAmendment[i], df.title[i]))
            subjects_dict['sponsor'].append(get_sponsor(df.isAmendment[i], df.title[i]))

        count_subjects_dict = {}

        for i in range(0, len(tmp)):
            if type(tmp[i]) == list:        
                for j in range(0, len(tmp[i])):
                    if tmp[i][j] in count_subjects_dict:
                        count_subjects_dict[tmp[i][j]] += 1
                    else:
                        count_subjects_dict[tmp[i][j]] = 1
#         subjects_dict = {}
        for key in count_subjects_dict:
            subjects_dict[key] = find_subject_in_bills(key)
        

        subjects_df = pd.DataFrame(data=subjects_dict)

        final = pd.concat([df, subjects_df], axis=1)
        final.to_csv(path2, index=False)

nan
nan
